In [ ]:
!git clone https://github.com/openai/human-eval.git
!pip install -e human-eval
import sys
sys.path.append('/kaggle/working/human-eval')

In [ ]:
import torch
from transformers import pipeline
from human_eval.data import write_jsonl, read_problems
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from huggingface_hub import login

# Paste your Hugging Face token here
token = ""

login(token)

In [ ]:
model_id = "meta-llama/Llama-3.2-1B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

In [ ]:
i=0

In [ ]:
def split_and_trim_code(text: str):
    # Find the index where "def " starts
    def_index = text.find('def ')
    if def_index == -1:
        return ""  # return an empty string if "def " is not found

    # Extract the code starting from "def "
    code_after_def = text[def_index:]

    # Find the end of the "def " line
    def_line_end_index = code_after_def.find('\n')
    if def_line_end_index != -1:
        # Skip the entire line where "def " is found
        code_after_def = code_after_def[def_line_end_index+1:]
    
    # Find the index of "```" that comes after "def "
    end_code_index = code_after_def.find('```')
    
    # If "```" is found, return the code until just before "```"
    if end_code_index != -1:
        code = code_after_def[:end_code_index]
    else:
        # If no "```" is found, return the code until the end of the string
        code = code_after_def
    
    # Removing leading and trailing whitespaces and newlines
    return code.strip()

In [ ]:
def generate_one_completion(prompt):
    global i 
    i += 1
    print("Generating")
    print(i)

    # Wrapping the prompt with instruction format tokens
    p = f"WRITE THE FULL COMPLETE FUNCTION (EG WITH def ....) END CODE WITH '```'. NOTE YOU ABSOLUTELY MUST END THE CODE WITH END CODE WITH '```' OR ELSE THE CODE WILL NOT BE INTERPRETTED!!!! {prompt}"

    # Generate a response using the model
    messages = [
        {"role": "system", "content": "You are a friendly chatbot"},
        {"role": "user", "content": p}
    ]
    
    with torch.no_grad():
        outputs = pipe(
            messages,
            max_new_tokens=3500,
        )
    
    response = outputs[0]["generated_text"][-1]
    
    # Print the text split and trimmed
    print("###" * 10)
    print(split_and_trim_code(response['content']))
    return split_and_trim_code(response['content'])

In [ ]:
problems = read_problems()

# Define the number of samples to generate for each problem
num_samples_per_task = 10

# Generate samples
samples = [
    dict(task_id=task_id, completion=generate_one_completion(problem["prompt"]))
    for task_id, problem in problems.items()
    for _ in range(num_samples_per_task)
]

# Save generated samples in jsonl format
write_jsonl("/kaggle/working/llama_samples_10.jsonl", samples)